## Classify Flooding in Imagery with Python in ArcGIS Pro 

As a remote sensing analyst in a humanitarian organization, you conduct analyses to support responses to disasters. For flood events, you must quickly analyze satellite imagery and identify areas that have been recently flooded.

In this lesson, you will learn how to develop an automated tool to identify flooded areas in satellite imagery. You will first use Python code running in a notebook in ArcGIS Pro. Then you'll turn the code into a script tool that can be used by analysts who don't have a programming background. 

### Import required Python modules and packages
You'll import the following modules and packages used in the process.
- [os](https://docs.python.org/3/library/os.html): A standard Python module for operating system-dependent functions, such as path manipulation.
- [glob](https://docs.python.org/3/library/glob.html): A standard Python module that enables simple pathname matching and list generation.
- [arcpy](https://pro.arcgis.com/en/pro-app/latest/arcpy/main/arcgis-pro-arcpy-reference.htm): A Python site package installed with ArcGIS Pro to perform _geographic data analysis_, _data conversion_, _data management_, and _map automation_.
- arcpy.sa: A Python module within the arcpy package for analyzing raster and vector data with the ArcGIS [Spatial Analyst extension](https://pro.arcgis.com/en/pro-app/2.8/arcpy/spatial-analyst/what-is-the-spatial-analyst-module.htm). You must have a Spatial Analyst license to use this module.  

This workflow uses packages that are installed with ArcGIS Pro's Python 3 distribution, so you will not need to install additional packages.

1. Run the cell below to import the required modules and packages.

In [ ]:
# Imports
import os
from glob import glob
import arcpy
from arcpy.sa import *

The modules and packages are now available for your code to use.

### Set paths to the input and output folders
To run the code, you must set three variables to store the paths to the input imagery and to the location where the results will be saved.  
The variables are:
- before_img_folder: This is the path to the folder on your hard drive where the pre-flood/before imagery is located.
- after_img_folder: This is the path to the folder on your hard drive where the post-flood/after imagery is located.
- final_output_folder: This is the path to the folder on your hard drive where the final resulting output files are written.

1. If you used a path other than **C:\Lessons\AnalyzeFlooding\Sentinel_2_Clipped** for the data, you must edit the three paths in the cell below before running the cell.  

If you edit the paths, make sure the new paths include the **Before**, **After**, and **Output** folder names.

2. Run the cell below to create the variables and set them to hold the input and output data paths.  

In [ ]:
# Set the input and output folder location variables.
# These must point to the locations of the folders in the Sentinel_2_Clipped folder you extracted.
# The "r" prefixes to the path strings indicate that these are "raw" strings in Python, and 
# the \ characters should not be treated as an escape character. If you edit these paths, remember
# to keep the \Before, \After, and \Output folder names in the paths.

# This is the folder containing the pre-flood imagery
before_img_folder = r"C:\Lessons\AnalyzeFlooding\Sentinel_2_Clipped\Before"

# This is the folder containing the post-flood imagery
after_img_folder = r"C:\Lessons\AnalyzeFlooding\Sentinel_2_Clipped\After"

# This is the folder where the final result files will be written
final_output_folder = r"C:\Lessons\AnalyzeFlooding\Sentinel_2_Clipped\Output"

### Create a function to identify imagery band data  

Your analysis will use Sentinel-2 multispectral imagery. These images are composed of [13 spectral bands](https://sentinels.copernicus.eu/web/sentinel/technical-guides/sentinel-2-msi/msi-instrument), but in this workflow you will only use 6 specific bands: Blue, Green, Red, Red Edge 1, Near Infrared 2, and Shortwave Infrared 2. The bands are stored as separate files.

Before you can begin the analysis, you must create a function to identify the files for the spectral bands from the imagery, and store the paths to those files in variables. This will allow you to refer to the bands using short variable names, rather than the long file paths. Sentinel-2 imagery is typically delivered with each of the bands stored in a separate JPEG 2000 (.jp2) file. Each band file ends with the band number as part of the file name. For example, the red band, band 4, file names end with "B04.jp2". With this formatting in mind, you will create a Python function that will take the folder containing the band images as an input parameter and return a variable that references each band's absolute path on your computer.

For the analysis you will use four specific bands, Red-Edge-1, SWIR-2, Green, and Near-Infrared-2. You will also use the Red and Blue bands to generate a composite image to visually inspect the imagery. This process can be used to access any of the bands to create other imagery analysis indicies.

To do this, you will define a function `create_sen2_band_variables` to do the following:  
- Get a list of all .jp2 image files in the input folder using the glob module and store that list in a variable named `band_list`.
- Use [list comprehension](https://docs.python.org/3/tutorial/datastructures.html#list-comprehensions) to find the files corresponding with the various Sentinel-2 bands, and store their paths in variables with their respective names such as `Blue`, `Green`, `Red`, or `NIR`.
- Return the band variables containing the paths to the image files.

1. Run the cell below to define the function to create the band path variables.

In [ ]:
def create_sen2_band_variables(in_folder):
    """A function that creates band variables for Sentinel-2 imagery given the folder with all the band images."""
    
    # Use arcpy.AddMessage like print() to print to the ArcGIS Geoprocessing messages
    arcpy.AddMessage("Creating variables for image bands...")

    # Get a list of the jpg2000 files in the input folder and store it in a list
    band_list = glob(in_folder + "/*.jp2")

    # Use list comprehension to get files in the band_list which correspond to the specific Sentinel-2 band file names
    Blue = [x for x in band_list if x.endswith("B02.jp2")][0]
    Green = [x for x in band_list if x.endswith("B03.jp2")][0]
    Red = [x for x in band_list if x.endswith("B04.jp2")][0]
    Red_Edge_1 = [x for x in band_list if x.endswith("B05.jp2")][0]
    NIR = [x for x in band_list if x.endswith("B08.jp2")][0]
    SWIR2 = [x for x in band_list if x.endswith("B12.jp2")][0]

    # Return the band variables
    return Blue, Green, Red, Red_Edge_1, NIR, SWIR2

Now that the function has been defined, you will test it by calling it on an image folder to see if it creates the band image paths correctly.  
Since the function returns data for several bands, you will set the results of the function equal to a corresponding set of variables. The order of the variables you create must match the order of the band data returned by the function. In this case, you will run the function on the post-flood imagery. To indicate this, you'll add an "after" prefix to each variable name to distinguish it from the band data for the pre-flood imagery.  

2. Run the cell below to test the function.

In [ ]:
# call the create_sen2_band_variables function on the folder containing the after imagery
after_Blue, after_Green, after_Red, after_Red_Edge_1, after_NIR, after_SWIR2 = create_sen2_band_variables(after_img_folder)

# print a couple bands to see the results
print(after_Red)  # print the path of the Red band
print(after_NIR)  # print the path of the NIR band

If you see two paths printed after the previous cell, such as the following, then the function correctly selected band 4 for the Red band and band 8 for the NIR band.  
    C:\Lessons\AnalyzeFlooding\Sentinel_2_Clipped\After\Clipped_T42TVK_20200504T061629_B04.jp2
    C:\Lessons\AnalyzeFlooding\Sentinel_2_Clipped\After\Clipped_T42TVK_20200504T061629_B08.jp2  
    
The paths are the paths to the band images on your computer.  
If you see an error message such as **IndexError: list index out of range**, go back and check the cell where you set the paths to the Before, After, and Output folders on your computer. There is probably a problem with these paths. Check the paths on your computer and edit the paths in the cell to match, run the cell again, and run the cell to test the function again.

Next, you will view the post-flood imagery you are working with. To do so, you will create a composite image using a false-color band combination (Near Infrared, Red, Green) to make the water and land more distinct. You will use the [Composite Bands geoprocessing tool](https://pro.arcgis.com/en/pro-app/latest/tool-reference/data-management/composite-bands.htm) to accomplish this. 

3. Run the cell below to draw the composite image.  

When you run the cell, it may take some time to process. After it finishes, a message will appear telling you that the tool succeeded.

In [ ]:
# create the composite image, storing the output in memory
arcpy.CompositeBands_management(in_rasters=f"{after_NIR};{after_Red};{after_Green}",
                                out_raster=r"memory\after_composite_img")

4. Click the **Map** tab to view the results. 

5. In the **Contents** pane, right-click **after_composite_img** and click **Zoom To Layer**.  

You see the False Color Infrared image representing the post flood area. Vegetated areas appear red and water appears vibrant blue.

### Create spectral index functions

To identify water in the imagery, you will use spectral indices. A spectral index applies a mathematical calculation to compute a ratio between different bands for every pixel in the imagery, with the goal of highlighting a specific phenomenon. You will use two indices, both designed for water identification:  [Sentinel-2 Water Index](https://www.mdpi.com/2073-4441/13/12/1647/htm) and the [Normalized Difference Water Index](https://pro.arcgis.com/en/pro-app/2.8/arcpy/spatial-analyst/ndwi.htm) (NDWI).

- The SWI relies on the Red Edge 1 and SWIR2 bands. The formula for this index is: `SWI = (Red_Edge1 - SWIR2) / (Red_Edge1 + SWIR2)`

- The NDWI uses the Green and Near Infrared (NIR) bands. The formula for this index is: `NDWI = (Green - NIR) / (Green + NIR)`

The overall strategy for your analysis will be to do the following:

- Apply SWI to find the pixels in the imagery that are covered with water. 

- Apply NDWI to find the pixels  in the imagery that are covered with water. 

- Find the pixels where water has been detected for both SWI and NDWI.  

This approach ensures more robust detection results than using a single index.  
**Note**: While you will only use two indices for this exercise, including more indices in this process can increase the accuracy of the analysis.  

You will use the `arcpy.sa` module to do [Map Algebra](https://pro.arcgis.com/en/pro-app/latest/arcpy/spatial-analyst/an-overview-of-the-map-algebra-operators.htm) to calculate the cell values. There are also a number of built-in spectral index [raster functions](https://pro.arcgis.com/en/pro-app/latest/arcpy/image-analyst/an-overview-of-the-image-analyst-functions.htm) available in the [Image Analysis module](https://pro.arcgis.com/en/pro-app/latest/arcpy/image-analyst/what-is-the-image-analyst-module.htm). 

1. Run the cell below to define the swi_processor function.

In [ ]:
# creates SWI processor function
def swi_processor(red_edge1_band, swir2_band):
    """Create a function which calculates the SWI for the given input image."""
    
    arcpy.AddMessage("\nBegining SWI Calculation...")
    
    # Calculate the SWI - Sentinel-2 Water Index
    # SWI Formula = (Red_Edge1 - SWIR2) / (Red_Edge1 + SWIR2)
    
    # Create a variable to store the calculation for the numerator
    # using arcpy spatial analyst Float tool to create a raster with floating point cell values
    Numerator = arcpy.sa.Float(Raster(red_edge1_band) - Raster(swir2_band))
    
    # Create a variable to store the calculation for the denominator
    # using arcpy spatial analyst Float tool to create a raster with floating point cell values
    Denominator = arcpy.sa.Float(Raster(red_edge1_band) + Raster(swir2_band))
    
    # Use the arcpy spatial analyst Divide tool to divide the numerator and the denominator 
    SWI = arcpy.sa.Divide(Numerator, Denominator)

    # return the results
    return SWI
    
    arcpy.AddMessage("SWI Successfully Generated")

Next, you will create another function for the NDWI index. It will use the same basic workflow and process as the SWI function but will use the NDWI formula and required bands instead.

2. Run the cell below to define the ndwi_processor function.

In [ ]:
# creates NDWI processor function
def ndwi_processor(green_band, nir_band):
    """Create a function which calculates the NDWI for the given input image."""

    arcpy.AddMessage("\nBegining NDWI Calculation...")
    
    # Calculate the NDWI - Normalized Difference Water Index
    # NDWI Formula = (Green - NIR) / (Green + NIR)

    # Create a variable to store the calculation for the numerator
    # using arcpy spatial analyst Float tool to create a raster with floating point cell values
    Num = arcpy.sa.Float(Raster(green_band) - Raster(nir_band))
    
    # Create a variable to store the calculation for the denominator
    # using arcpy spatial analyst Float tool to create a raster with floating point cell values
    Denom = arcpy.sa.Float(Raster(green_band) + Raster(nir_band))
    
    # Use the arcpy spatial analyst Divide tool to divide the numerator and the denominator 
    NDWI = arcpy.sa.Divide(Num, Denom)
    
    # return the results
    return NDWI
    
    arcpy.AddMessage("NDWI Successfully Generated")

Now that the two functions are defined, you'll run them on the image and view the results on the map.  

3. Run the cell below to calculate the SWI and NDWI indices for the after-flooding image.

In [ ]:
# Process SWI 
# Create the SWI raster
after_swi_calc = swi_processor(red_edge1_band=after_Red_Edge_1,
                               swir2_band=after_SWIR2)

# Create path for output SWI file
after_swi_raster = r"memory\after_swi_raster"

# Save the result
after_swi_calc.save(after_swi_raster)


# Process NDWI 
# Create the NDWI raster
after_ndwi_calc = ndwi_processor(green_band=after_Green,
                                 nir_band=after_NIR)

# Create path for output NDWI file
after_ndwi_raster = r"memory\after_ndwi_raster"

# Save the result
after_ndwi_calc.save(after_ndwi_raster)

Now that you've added the index rasters to the map, you can inspect the images.  

4. Click the **Map** tab to view the results.  

Areas with higher pixel values correspond to water and are symbolized by white.  

Can you see the areas that correspond with water?

### Create a thresholding function

Now that you have created the functions to calculate the indices, you must determine how to separate the water from the non-water in the resulting SWI and NDWI rasters. A common way to approach this problem is using histogram thresholding. This technique attempts to determine a single threshold value that categorizes and splits the raster’s pixel values. This classifies all of the pixels as either non-water or water, expressed as either 0 or 1. 

This is often done by manual trial and error, where the analyst visually interprets a histogram and chooses what they think is the best threshold value. Other techniques use various statistical and mathematical tests to determine a threshold value. The ArcPy Spatial Analyst module contains the [Threshold function](https://pro.arcgis.com/en/pro-app/2.8/arcpy/spatial-analyst/threshold.htm), which uses the [Otsu method](https://pro.arcgis.com/en/pro-app/2.8/help/analysis/raster-functions/binary-thresholding-function.htm) to automatically determine the optimal threshold value for a binary classification of a raster dataset.

To process the thresholds, you will create another function. Inside the function, the `arcpy.sa.Threshold()` function runs on the input index raster.

1. Run the cell below to define the create_threshold_raster function.

In [ ]:
def create_threshold_raster(in_raster):
    """Creates a function that thresholds the input raster, and then returns it."""
    
    # Run the Ostu thresholding function on the input raster
    thresh_calc = arcpy.sa.Threshold(in_raster)
    
    # Return the result
    return thresh_calc

Now that the function is defined, you'll use it to generate the thresholds for the NDWI and SWI rasters.

2. Run the cell below to apply the create_threshold_raster function to the two index rasters.

In [ ]:
# Process NDWI 
# Use the threshold_raster function to process the NDWI raster
after_ndwi_thresh_calc = create_threshold_raster(in_raster=after_ndwi_raster)

# Create a path for output NDWI file
after_ndwi_thresh_raster = r"memory\after_ndwi_thresh"

# Save the NDWI thresh raster
after_ndwi_thresh_calc.save(after_ndwi_thresh_raster)

# Create a raster layer for the NDWI threshold file so it can be viewed in the results map tab
after_ndwi_threshold_layer = arcpy.MakeRasterLayer_management(after_ndwi_thresh_raster, 'after_ndwi_threshold')


# Process SWI 
# Use the threshold_raster function to process the SWI raster
after_swi_thresh_calc = create_threshold_raster(in_raster=after_swi_raster)

# Create a path for output NDWI file
after_swi_thresh_raster = r"memory\after_swi_thresh"

# Save the SWI thresh raster
after_swi_thresh_calc.save(after_swi_thresh_raster)

# Create a raster layer for the SWI threshold file so it can be viewed in the results map tab
after_swi_threshold_layer = arcpy.MakeRasterLayer_management(after_swi_thresh_raster, 'after_swi_threshold')

Now that you've run the create_threshold_raster function on the index rasters, you can inspect the two thresholded rasters.  

3. Click the **Map** tab to view the results.  

Areas with pixel values of 1 are classified as water. Areas with values of 0 are non-water.

Can you see the areas that are classified as water? How do these areas differ in the two thresholded output layers?

Higher values in the index raster were assigned a threshold value of 1. This is convenient in this analysis, because the index values corresponding to water were closer to 1. However, you may need to be careful if you add other indices or use this approach to identify different types of features. Not all indices classify the features of interest as high pixel values.  

**Note**: Whenever a layer is added to the map, it will be symbolized with a random color or color ramp. The randomly selected colors are not always the best choice for visual interpretation. You can set your own color for the layer from the Contents pane. Right-click the color square under the layer, and choose a different color. You can do this for any layer that is added to the map.

### Extract the water areas and create a confidence raster

You will add the threshold rasters together to compare the results of the thresholding steps. The resulting raster will have three possible values:

| Pixel value | Description |
| ----------- | ----------- |
| 0 | Pixels where neither of the indices identified water |
| 1 | Pixels where only one index identified water |
| 2 | Pixels where both indices identified water |

The result can be thought of as a confidence raster. If both indices indicate that a pixel is water, you have greater confidence in the result than if only one index classified it as water.  

To add the two rasters together, you must create [raster objects](https://pro.arcgis.com/en/pro-app/latest/arcpy/classes/raster-object.htm) by using the `Raster()` from the threshold files and then add them together with the + operator. You will store this process in a function that can be called again later.

1. Run the cell below to define the create_water_confidence_raster function.

In [ ]:
def create_water_confidence_raster(ndwi_threshold_raster, swi_threshold_raster):
    """Create a function that calculates the sum of two rasters."""
    
    # Add the two threshold rasters together by creating raster objects of each and combining them using the addition + operator
    water_confidence_raster = Raster(ndwi_threshold_raster) + Raster(swi_threshold_raster)

    # Return the result
    return water_confidence_raster

Now that you have defined the create_water_confidence_raster function, you'll use it to create the confidence raster.  

2. Run the cell below to create the confidence raster.

In [ ]:
# Call the create_water_confidence_raster function
after_water_confidence_raster = create_water_confidence_raster(ndwi_threshold_raster=after_ndwi_thresh_raster, 
                                                         swi_threshold_raster=after_swi_thresh_raster)

# Create a path for output water confidence raster file
after_water_confidence_raster_path = r"memory\after_water_confidence_raster"

# Save the water confidence raster to a file in memory
after_water_confidence_raster.save(after_water_confidence_raster_path)

# Create a raster layer for the confidence raster file so it can be viewed in the results map tab
# after_water_confidence_matrix_layer = arcpy.MakeRasterLayer_management(after_water_confidence_raster, 'after_water_confidence_matrix')

Now that the confidence matrix layer has been added to the map, you can explore the results.  

3. Click the **Map** tab to view the results.  

Areas with pixel values of 2 were classified as water by both indices.  
Areas with values of 1 were classified as water by only one of the indices.  
Areas with pixel values of 0 were classified as water by neither of the indices.  

You can turn the layers on and off and compare the confidence raster with the index rasters, and with the composite image.

### Extract the high confidence values  

The last step in the process of extracting the water is to reclassify the confidence raster to set all pixels other than the high confidence (2) values to 0. Pixel values of 1, areas of only moderate confidence, will be reset to 0 values.

You will use the [Reclassify](https://pro.arcgis.com/en/pro-app/latest/tool-reference/spatial-analyst/reclassify.htm) geoprocessing tool in the Spatial Analyst toolbox. 

First, you'll define the [remap values](https://pro.arcgis.com/en/pro-app/latest/arcpy/spatial-analyst/remapvalue-class.htm). The Reclassify tool uses the remap values to determine which values you want to change and what to change them to.  

Sometimes you'll want to change a range of values to a new value. In these cases, you can use [Remap Range](https://pro.arcgis.com/en/pro-app/latest/arcpy/spatial-analyst/an-overview-of-transformation-classes.htm) to change ranges of values.

1. Run the cell below to create the remap values and reclassify confidence raster.

In [ ]:
# Create the remap values to set any pixels with the value of 1 to 0.
remap_value = RemapValue([[1, 0]])

# Reclassify the water mask
after_water_mask_reclass = Reclassify(in_raster=after_water_confidence_raster_path, 
                                      reclass_field="value", 
                                      remap=remap_value)

# Create a path for output water mask file
after_water_mask_raster = r"memory\after_water_mask_high_confidence"

# Save the water mask
after_water_mask_reclass.save(after_water_mask_raster)

# Create a raster layer so it can be viewed in the results map tab.
#after_water_mask_layer = arcpy.MakeRasterLayer_management(after_water_mask_reclass, 'after_water_mask_high_confidence')

2. Click the **Map** tab to view the results.  

The areas where both indices agreed that there was water (pixel value = 2) contrast with the areas where they disagreed, or agreed that there was no water (values = 0).

### Process the before image  

Now that all of the functions for your process have been created, and run on the post-flood imagery, you will re-use those functions for the pre-flood image.  
Because you understand the steps involved, you can put the code in one code block, and run it all at the same time.

The before image folder path was set early in the notebook and stored as the variable `before_img_folder`.


1. Run the cell below to run all of the processing steps on the 'before' image. 

In [ ]:
# Step 1: create the band variables 
before_Blue, before_Green, before_Red, before_Red_Edge_1, before_NIR, before_SWIR2 = create_sen2_band_variables(
    in_folder=before_img_folder)


# Step 2: Create a false-color infrared composite image 
# create a composite image, storing the output in memory
arcpy.CompositeBands_management(in_rasters=f"{before_NIR};{before_Red};{before_Green}",
                                out_raster=r"memory\before_composite_img")

# Step 3: create indices 
# create the SWI raster
before_swi_calc = swi_processor(red_edge1_band=before_Red_Edge_1,
                                swir2_band=before_SWIR2)

# create path for output NDWI file
before_swi_raster = r"memory\before_swi_raster"

# Save result to file
before_swi_calc.save(before_swi_raster)

# create a raster layer for the NDWI threshold file so it can be viewed in the results map tab.
before_swi_layer = arcpy.MakeRasterLayer_management(before_swi_raster, 'before_swi')

# create the NDWI raster
before_ndwi_calc = ndwi_processor(green_band=before_Green,
                                  nir_band=before_NIR)

# create path for output NDWI file
before_ndwi_raster = r"memory\before_ndwi_raster"

# Save result to file
before_ndwi_calc.save(before_ndwi_raster)

# create a raster layer for the NDWI threshold file so it can be viewed in the results map tab.
before_ndwi_layer = arcpy.MakeRasterLayer_management(before_ndwi_raster, 'before_ndwi')


# Step 4: threshold indices 
# Threshold for NDWI
before_ndwi_thresh_calc = create_threshold_raster(in_raster=before_ndwi_raster)

# create path for output NDWI file
before_ndwi_thresh_raster = r"memory\before_ndwi_thresh"

# save the NDWI thresh raster
before_ndwi_thresh_calc.save(before_ndwi_thresh_raster)

# create a raster layer for the NDWI threshold file so it can be viewed in the results map tab.
before_ndwi_threshold_layer = arcpy.MakeRasterLayer_management(before_ndwi_thresh_raster, 'before_ndwi_threshold')

# Threshold for SWI
before_swi_thresh_calc = create_threshold_raster(in_raster=before_swi_raster)

# create path for output SWI file
before_swi_thresh_raster = r"memory\before_swi_thresh"

# save the SWI thresh raster
before_swi_thresh_calc.save(before_swi_thresh_raster)

# create a raster layer for the SWI threshold file so it can be viewed in the results map tab.
before_swi_threshold_layer = arcpy.MakeRasterLayer_management(before_swi_thresh_raster, 'before_swi_threshold')


# Step 5: calculate confidence raster 
# create water confidence raster
before_water_confidence_raster = create_water_confidence_raster(ndwi_threshold_raster=before_ndwi_thresh_raster,
                                                                swi_threshold_raster=before_swi_thresh_raster)

# create path for output water confidence matrix file
before_water_confidence_raster_path = r"memory\before_water_confidence_raster"

# save the water confidence matrix to a file in memory
before_water_confidence_raster.save(before_water_confidence_raster_path)

# create a raster layer for the NDWI threshold file so it can be viewed in the results map tab.
# before_water_confidence_layer = arcpy.MakeRasterLayer_management(before_water_confidence_raster, 'before_water_confidence_raster')

# Step 6: extract water pixels 
# create the remap value to set any pixels with the value of 1 to 0.
remap_value = RemapValue([[1, 0]])

# reclassify the water mask
before_water_mask_reclass = Reclassify(in_raster=before_water_confidence_raster_path,
                                       reclass_field="value",
                                       remap=remap_value)

# create path for output water mask file
before_water_mask_raster = r"memory\before_water_mask_high_confidence"

# save the water mask to
before_water_mask_reclass.save(before_water_mask_raster)

2. Click the **Map** tab to view the results.  

The before_water_mask_reclass layer shows the areas that were covered with water before the flood.

3. Press the _Ctrl_ key while clicking the box beside the before_water_mask_reclass layer to turn all the layers off.  
4. Check the before_composite_img layer box to turn the layer on. 
5. Check the after_composite_img layer box to turn the layer on.  

You can visually compare the two composite images to see the change that occurred by turning the upper layer, before_composite_img, on and off.

### Analyze the change between the before and after water areas  

Now you will determine which areas from the after image are flood waters, and not pre-existing water bodies. This is called a change analysis. To do this, you'll calculate the difference between the two water mask rasters, subtracting the pre-flood water mask from the post-flood water mask.

1. Run the cell below to subtract the pre-flood water mask from the post-flood water mask, and reclassify the results.

In [ ]:
# Using raster objects, subtract the before water mask from the after water mask
flooded_area_calc = Raster(after_water_mask_reclass) - Raster(before_water_mask_reclass)

# Createa path for flooded_area_calc
flooded_area_calc_raster = r"memory\flooded_area_calc"

# Save the flooded_area_calc raster
flooded_area_calc.save(flooded_area_calc_raster)

The result raster contains three pixel values:

| Pixel value | Description | Importance |
| ----------- | ----------- | ---------- |
| 2 | Where there was water only in the after image | **Important** |
| 0 | Where there was water before and after, or where there was no water in either | Not important |
| -2 | Where there was no water in the after image, and water in the before image | Not important |

The pixels with a value of 2 are the areas that are newly flooded, and did not contain water in the pre-flood imagery. Only these values are of interest, so you can reclassify this raster to set all other pixels to "NoData".

2. Run the cell below to reclassify the raster to only show the newly flooded areas.

In [ ]:
# Reclassify the final flood area 

# Create the remap values lists
remap_value_final = RemapValue([[-2, "NoData"], [0, "NoData"]])

# Reclassify the water mask
flooded_area_final = Reclassify(in_raster=flooded_area_calc, 
                                reclass_field="value", 
                                remap=remap_value_final)

3. Click the **Map** tab to view the results. 
4. Uncheck the flooded_area_calc layer.  
5. Compare the flooded_area_final layer to the two composite images.

### Save your analysis results  

Now that you've found the newly flooded areas, you can save them for later use.  

All of the results so far have been saved to a temporary memory workspace that will be removed when you close ArcGIS Pro. However, you will want these final results for a report and further analysis. You will save the result raster and a polygon feature class to the `final_output_folder` that was stored as a variable at the beginning of this lesson.

1. Run the cell below to save the flooded_area_final raster as a .tif file in the final_output_folder.

In [ ]:
# Create path for output flooded area tif file
flooded_area_final_raster = os.path.join(final_output_folder, "Flooded_Area_Final_Raster.tif")

# Save the final flooded area to a tif file
flooded_area_final.save(flooded_area_final_raster)

You'll use the [Raster to Polygon](https://pro.arcgis.com/en/pro-app/2.8/tool-reference/conversion/raster-to-polygon.htm) tool to save a copy of the results as shapefile.  
2. Run the cell below to convert the raster to a polygon shapefile in the final_output_folder.

In [ ]:
# Create path for output flooded area polygon file
flooded_area_final_poly = os.path.join(final_output_folder, "Flooded_Area_Final_Poly.shp")

# Convert to polygon
arcpy.RasterToPolygon_conversion(in_raster=flooded_area_final, 
                                 out_polygon_features=flooded_area_final_poly, 
                                 simplify="NO_SIMPLIFY", 
                                 raster_field="Value")

You've completed the analysis. The flooded areas are stored in an image file and a shapefile on your hard drive, ready to use in a report or a further analysis.

#  Create a script tool from your code  

Now that you've finished the analysis, you will create a script tool from your code, so others can use it. Open and follow the instructions in the **Part_2_Script_Tool_Creation.ipynb** file to create a script tool.

#  Clean up temporary layers

Since you don't need the temporary layers, you can remove them from the map. One way to do this is manually, by right-clicking individual layers and clicking **Remove**. You can also remove all the temporary layers using Python code.

1. Run the cell below to remove temporary layers from the map.

In [ ]:
# Get the currently open ArcGIS Pro Project
aprx = arcpy.mp.ArcGISProject("current")
# Get the map
m = aprx.listMaps("Map")[0]
# Get a list of the layers on the map
thelyrs = m.listLayers()
# Check each layer to determine if it is a temporary, in memory layer, 
# and remove the layer if it is.
for lyr in thelyrs:
    print(lyr.dataSource)
    if "INSTANCE_ID=GPProMemoryWorkspace" in lyr.dataSource:
        print("Removing: ", lyr.name )
        m.removeLayer(lyr)